# Reflection Pattern

We are going to implement is the **reflection pattern**. 

---

<!-- <img src="../img/reflection_pattern.png" alt="Alt text" width="600"/> -->

---

This pattern allows the LLM to reflect and critique its outputs, following the next steps:

1. The LLM **generates** a candidate output. If you look at the diagram above, it happens inside the **"Generate"** box.
2. The LLM **reflects** on the previous output, suggesting modifications, deletions, improvements to the writing style, etc.
3. The LLM modifies the original output based on the reflections and another iteration begins ...



## Generation Step

**We are going to build it from scratch using no framework.**

We are using gpt-4o for now. 
The task we will want to have completed by the Reflection Agent is:
"Generate a Python implementation of the Bubble Sort algorithm"


### Instantiate Client and relevant imports

In [3]:
import os
from pprint import pprint
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display_markdown

# Remember to load the environment variables. You should have the Groq API Key in there :)
load_dotenv()

client = OpenAI()

First we define the system prompt for the **"generation"** chat history. We mention that the LLM is  eager to receive feedback or critique by the user.

In [4]:
generation_chat_history = [
    {
        "role": "system",
        "content": "You are a Python programmer tasked with generating high quality Python code."
        "Your task is to Generate the best content possible for the user's request. If the user provides critique," 
        "respond with a revised version of your previous attempt."
    }
]

Now, as the user, we are going to ask the LLM to generate an implementation of the **Bubble Sort** algorithm. Just add a new message with the **user** role to the chat history.

In [5]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of the Bubble Sort algorithm"
    }
)

Let's generate the first version of the essay.

In [7]:
bubblesort_code = client.chat.completions.create(
    messages=generation_chat_history,
    model="gpt-4o"
).choices[0].message.content

generation_chat_history.append(
    {
        "role": "assistant",
        "content": bubblesort_code
    }
)

APIConnectionError: Connection error.

In [ ]:
display_markdown(bubblesort_code, raw=True)

## Reflection Step

Now, let's allow the LLM to reflect on its outputs by defining another system prompt. This system prompt will tell the LLM to act as Andrej Karpathy, computer scientist and Deep Learning wizard.

In [8]:
reflection_chat_history = [
    {
    "role": "system",
    "content": "You are Andrej Karpathy, an experienced computer scientist. You are tasked with generating critique and recommendations for the user's code",
    }
]

The user message, in this case,  is the essay generated in the previous step. We simply add the `bubblesort_code` to the `reflection_chat_history`.

In [ ]:
reflection_chat_history.append(
    {
        "role": "user",
        "content": bubblesort_code
    }
)

Now, let's generate a critique to the Python code.

In [ ]:
critique = client.chat.completions.create(
    messages=reflection_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

In [ ]:
display_markdown(critique, raw=True)

Finally, we just need to add this *critique* to the `generation_chat_history`, in this case, as the `user` role.

In [ ]:
generation_chat_history.append(
    {
        "role": "user",
        "content": critique
    }
)

## Generation Step (II)

In [ ]:
essay = client.chat.completions.create(
    messages=generation_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

In [ ]:
display_markdown(essay, raw=True)

## And the iteration starts again ...

After **Generation Step (II)** the corrected Python code will be received, once again, by Karpathy. Then, the LLM will reflect on the corrected output, suggesting further improvements and the loop will go, over and over for a number **n** of total iterations.

> There's another possibility. Suppose the Reflection step can't find any further improvement. In this case, we can tell the LLM to output some stop string, like "OK" or "Good" that means the process can be stopped. However, we are going to follow the first approach, that is, iterating for a fixed number of times.

## Implementing a class 

Now that you understand the underlying loop of the Reflection Agent, let's implement this agent as a class.

In [ ]:
from agentic_patterns import ReflectionAgent

In [ ]:
agent = ReflectionAgent()

In [ ]:
generation_system_prompt = "You are a Python programmer tasked with generating high quality Python code"

reflection_system_prompt = "You are Andrej Karpathy, an experienced computer scientist"

user_msg = "Generate a Python implementation of the Merge Sort algorithm"

In [ ]:
final_response = agent.run(
    user_msg=user_msg,
    generation_system_prompt=generation_system_prompt,
    reflection_system_prompt=reflection_system_prompt,
    n_steps=10,
    verbose=1,
)

## Final result

In [ ]:
display_markdown(final_response, raw=True)